In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing

In [2]:
df = pd.read_csv('/Users/charlieruan/projects_data/2022/ORIE3120_project/fhvhv/2022/fhvhv_tripdata_2022-02.csv')
# filter trips that start or end in unknown location
df = df[(df['PULocationID'] < 264) & (df['DOLocationID'] < 264)]
df = df[df['hvfhs_license_num']=='HV0003']
# Since the most frequent trips are picked up and dropped off in the same location
# We will further filter so that the 2 locations are different 
df = df[df['PULocationID'] != df['DOLocationID']]

In [3]:
trip_freq = df.groupby(['PULocationID', 'DOLocationID']).size()
df_freq = pd.DataFrame(trip_freq)
df_freq = df_freq.reset_index()
# look at trips that end in financial district
df_freq_finan = df_freq[(df_freq['DOLocationID'] == 88) | (df_freq['DOLocationID'] == 87)]
df_freq_finan = df_freq_finan.sort_values(by=0, ascending=False)
df_freq_finan.head(20)

,PULocationID,DOLocationID,0
48379,231,87,4012
16268,79,87,3414
30581,148,87,2900
52545,249,87,2494
29619,144,87,2281
43802,211,87,2104
22912,114,87,1980
1884,13,87,1885
28156,138,87,1814
49125,234,87,1694


Based on the above dataframe, some intersting trips that we can look at are:
- (79, 87) East Village (where most bankers live at, where there are a lot of restaurants)
- (249, 87) West Village
- (138, 87) Laguardia
- (234, 87) Union Square, the furthest in Manhattan so far; also very specific
- (164, 87) Midtown South (where other banks are at)
- (230, 87) Time Square/Theatre District

Can pick two, one close and one far:
- (79, 87) East Village, roughly 10-15 minutes drive through FDR
- (230, 87) Time Square/Theatre District, 20 minutes when no traffic

### Filter, and get the travel time

In [4]:
# PUID = 79 # East Village
PUID = 230 # Time Square

In [5]:
df_EV = df[(df['PULocationID']==PUID) & (df['DOLocationID']==87)]
df_EV = df_EV[['pickup_datetime', 'dropoff_datetime']]

# convert to actual datetime object
df_EV['pickup_datetime'] = pd.to_datetime(df_EV['pickup_datetime'])
df_EV['dropoff_datetime'] = pd.to_datetime(df_EV['dropoff_datetime'])
df_EV.head()

,pickup_datetime,dropoff_datetime
6339,2022-02-01 00:14:38,2022-02-01 00:29:57
47346,2022-02-01 06:37:12,2022-02-01 07:00:56
67841,2022-02-01 07:15:14,2022-02-01 07:44:42
145328,2022-02-01 10:18:36,2022-02-01 10:47:41
164453,2022-02-01 11:01:07,2022-02-01 11:37:04


In [6]:
for index, row in df_EV.iterrows():
    df_EV.loc[index, 'date'] = row['pickup_datetime'].date()
    df_EV.loc[index, 'hr_of_day'] = row['pickup_datetime'].hour
    df_EV.loc[index, 'duration_min'] = (row['dropoff_datetime'] - 
                                        row['pickup_datetime']).total_seconds() / 60

In [7]:
df_avg_dur = df_EV.groupby(['date', 'hr_of_day']).mean()
df_avg_dur = df_avg_dur.reset_index()

In [8]:
# there is some entry that is empty, then we just fill that with the previous duration
dates = df_avg_dur.date.unique()
hours = np.arange(0, 24)
recent_dur = None
counter = 0

for date in dates:
    for hour in hours:
        df_cur = df_avg_dur[(df_avg_dur['date']==date) & (df_avg_dur['hr_of_day']==hour)]
        if len(df_cur) == 1:
            # there is an entry, then update most recent dur
            recent_dur = np.array(df_cur.duration_min)[0]
        else:
            # there is no such entry, we need to fill it in using most recent duration
            counter += 1
            df_avg_dur.loc[-counter] = [date, hour, recent_dur]

In [9]:
df_avg_dur = df_avg_dur.sort_values(by=['date', 'hr_of_day'])

# concatenate hour and date
for index, row in df_avg_dur.iterrows():
    cur_time = datetime.time(int(row['hr_of_day']), 0)
    df_avg_dur.loc[index, 'date_hr'] = datetime.datetime.combine(row['date'], cur_time)
df_avg_dur = df_avg_dur.set_index(df_avg_dur['date_hr'])

df_avg_dur

,date,hr_of_day,duration_min,date_hr
date_hr,,,,
2022-02-01 00:00:00,2022-02-01,0.0,15.316667,2022-02-01 00:00:00
2022-02-01 01:00:00,2022-02-01,1.0,15.316667,2022-02-01 01:00:00
2022-02-01 02:00:00,2022-02-01,2.0,15.316667,2022-02-01 02:00:00
2022-02-01 03:00:00,2022-02-01,3.0,15.316667,2022-02-01 03:00:00
2022-02-01 04:00:00,2022-02-01,4.0,15.316667,2022-02-01 04:00:00
...,...,...,...,...
2022-02-28 19:00:00,2022-02-28,19.0,29.066667,2022-02-28 19:00:00
2022-02-28 20:00:00,2022-02-28,20.0,20.757143,2022-02-28 20:00:00
2022-02-28 21:00:00,2022-02-28,21.0,19.510000,2022-02-28 21:00:00


In [10]:
df_train = df_avg_dur[df_avg_dur['date'] <= datetime.date(2022, 2, 20)]
df_test = df_avg_dur[df_avg_dur['date'] > datetime.date(2022, 2, 20)]

In [11]:
fitted = ExponentialSmoothing(df_train['duration_min'], trend = "add", seasonal = "mul", \
                              seasonal_periods = 24 * 7).fit()
fore = fitted.forecast(len(df_test))

/Users/charlieruan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/charlieruan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
/Users/charlieruan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/charlieruan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


In [15]:
plt.figure(figsize=(20, 8), dpi=100)
plt.plot(df_avg_dur['duration_min'], label='true values')
plt.plot(df_train['date_hr'], fitted.fittedvalues, label='fitted values')
fore.plot(label='forecasted')

# settings 
plt.xlabel('Time', fontsize=12)
plt.ylabel('Trip Duration (min)', fontsize=12)
plt.xlim(left=datetime.date(2022, 2, 1), right=datetime.date(2022, 2, 28))
plt.title('Exponential Smoothing on Trip Duration from Time Square to Financial District', fontsize=16, fontweight='bold')
plt.legend(fontsize=10);
plt.savefig("exp_smooth_trip_duration_timeSquare.png", dpi=300)
plt.clf();

<Figure size 2000x800 with 0 Axes>

In [16]:
loss = np.sum((fore - df_test['duration_min']).pow(2))
loss

4016.3962053694145